In [ ]:
from backend.app.core.supabase_client import get_supabase

supabase = await get_supabase()

# First get the transactions
gocardless_response = await supabase.table('gocardless_transactions')\
            .select('*')\
            .eq('user_id', "user_2tEnUq7rivacYtZnsAXPlC5gi9B")\
            .eq('coa_set_by', 'AI')\
            .execute()

gocardless_response.data

In [5]:
import requests
from dotenv import load_dotenv
import os
load_dotenv()

GOCARDLESS_CLIENT_ID = os.getenv('GOCARDLESS_CLIENT_ID')
GOCARDLESS_CLIENT_SECRET = os.getenv('GOCARDLESS_CLIENT_SECRET')
if not GOCARDLESS_CLIENT_ID or not GOCARDLESS_CLIENT_SECRET:
    raise ValueError('GOCARDLESS_CLIENT_ID and GOCARDLESS_CLIENT_SECRET must be set')
else:
    print('GOCARDLESS_CLIENT_ID and GOCARDLESS_CLIENT_SECRET are set')


''' STEP 1 - ACCESS TOKEN '''
# GoCardless OAuth endpoint for obtaining access tokens
AUTH_URL = 'https://bankaccountdata.gocardless.com/api/v2/token/new/'


def get_access_token():
    # Data to be sent in the POST request
    data = {
        "secret_id": GOCARDLESS_CLIENT_ID,
        "secret_key": GOCARDLESS_CLIENT_SECRET
    }

    # Making the POST request to obtain the access token
    response = requests.post(AUTH_URL, json=data)
    response.raise_for_status()  # Raise exception for non-200 responses

    # Parsing the JSON response
    access_token = response.json()
    return access_token

access_token = get_access_token()['access']

''' END USER SELECTING A BANK '''
"https://github.com/nordigen/nordigen-bank-ui"

url = "https://bankaccountdata.gocardless.com/api/v2/institutions/"
params = {
    "country": "PT"
}
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {access_token}"  # Replace ACCESS_TOKEN with your actual access token
}

list_of_banks = requests.get(url, headers=headers, params=params)

if list_of_banks.status_code == 200:
    list_of_banks = list_of_banks.json()
    print("Response:")
    print(list_of_banks)
else:
    print("Error:", list_of_banks.text)
    list_of_banks.raise_for_status()  # Raise exception for non-200 responses


GOCARDLESS_CLIENT_ID and GOCARDLESS_CLIENT_SECRET are set
Response:
[{'id': 'ABANCA_CAGLPTPL', 'name': 'Abanca', 'bic': 'CAGLPTPL', 'transaction_total_days': '90', 'countries': ['PT'], 'logo': 'https://cdn-logos.gocardless.com/ais/ABANCA_CAGLESMM.png', 'max_access_valid_for_days': '180'}, {'id': 'ABANCA_CORP_CAGLPTPL', 'name': 'Abanca Corporate', 'bic': 'CAGLPTPL', 'transaction_total_days': '90', 'countries': ['PT'], 'logo': 'https://cdn-logos.gocardless.com/ais/ABANCA_CAGLESMM.png', 'max_access_valid_for_days': '180'}, {'id': 'AIRWALLEX_AIPTAU32', 'name': 'Airwallex', 'bic': 'AIPTAU32', 'transaction_total_days': '730', 'countries': ['GB', 'EE', 'IT', 'ES', 'DE', 'CZ', 'PL', 'GR', 'HR', 'SI', 'CY', 'LV', 'IS', 'LU', 'DK', 'BE', 'HU', 'FI', 'FR', 'SE', 'AT', 'LT', 'BG', 'NO', 'SK', 'MT', 'NL', 'PT', 'LI', 'IE', 'RO'], 'logo': 'https://cdn-logos.gocardless.com/ais/AIRWALLEX_AIPTAU32_1.png', 'max_access_valid_for_days': '90'}, {'id': 'BANCOACTIVOBANK_ACTVPTPL', 'name': 'Banco ActivoBank',

In [ ]:
def extract_schema(json_list):
    """
    Extract schema from a list of dictionaries, showing all possible keys and their data types
    
    Args:
        json_list (list): List of dictionaries
    
    Returns:
        dict: Schema showing keys and their possible data types
    """
    schema = {}
    
    # Handle single dictionary case
    if isinstance(json_list, dict):
        json_list = [json_list]
    
    for item in json_list:
        for key, value in item.items():
            # Get the type of the value
            value_type = type(value).__name__
            
            # If key exists, add new type if different
            if key in schema:
                if value_type not in schema[key]:
                    schema[key].append(value_type)
            else:
                schema[key] = [value_type]
    
    return schema


In [ ]:
from app.services.gocardless import get_access_token, get_transactions
access_token = await get_access_token()
access_token = access_token['access']

accounts = ['a624f40e-6ecd-45e9-89d9-dd40d3cc7b7b']


transactions = await get_transactions(accounts, access_token)

transactions 

In [3]:
''' STEP 3 - CREATE END USER AGREEMENT '''
# API endpoint
url = "https://bankaccountdata.gocardless.com/api/v2/agreements/enduser/"

# Headers
headers = {
    "Authorization": f"Bearer {access_token}"
}

INSTITUTION_ID = "NATWEST_NWBKGB2L"
# Data
data = {
    "institution_id": INSTITUTION_ID,
    "max_historical_days": "730",
    "access_valid_for_days": "30",
    "access_scope": ["balances", "details", "transactions"]
}

# Make POST request
try:
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()  # Raise exception for non-200 responses
    result = response.json()
    print(result)
except requests.exceptions.RequestException as e:
    print(f"Error making POST request: {e}")

''' STEP 4 - BUILD A LINK '''
import random
def generate_random_id():
    return ''.join(random.choices('0123456789', k=21))
random_id = generate_random_id()

url = "https://bankaccountdata.gocardless.com/api/v2/requisitions/"
headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token}"  # Replace ACCESS_TOKEN with your actual access token
}
data = {
    #"redirect": "https://michaelalilondon.bubbleapps.io/version-test",
    "redirect": "http://localhost:3001/gocardless/callback",
    "institution_id": result['institution_id'],
    "reference": random_id,
    "agreement": result['id'],
    "user_language": "EN"
}
response = requests.post(url, headers=headers, json=data)
response.json()

NameError: name 'access_token' is not defined

In [ ]:
""" ONCE USER HAS BEEN REDIRECTED TO OUR SITE, WE CAN FETCH THEIR ACCOUNTS """
''' Fetch accounts for given requisition '''
req = "ece1a3fe-0db2-409b-8ab8-c2bdb1897761"
url = f"https://bankaccountdata.gocardless.com/api/v2/requisitions/{req}"

response = requests.get(url, headers=headers)
response.json()

""" THEN LOOP THROUGH ACCOUNTS LIST AND GET TRANSACTIONS FOR EACH ACCOUNT """
accounts = response.json()['accounts']

for account in accounts:
    url = f"https://bankaccountdata.gocardless.com/api/v2/accounts/{account}/transactions/"
    accounts_transactions = requests.get(url, headers=headers)




In [58]:
def transform_transactions(transactions):
    for transaction in transactions:
        # Extract currency and amount from transactionAmount
        currency = transaction['transactionAmount']['currency']
        # Convert amount to integer (assuming amount is a string like "10.50")
        amount_str = transaction['transactionAmount']['amount']
        # Remove decimal point and convert to integer (e.g., "10.50" becomes 1050)
        amount = int(float(amount_str) * 100)
        
        # Add new fields
        transaction['currency'] = currency
        transaction['amount'] = amount
        
        # Remove original transactionAmount field
        del transaction['transactionAmount']
    
    return transactions

In [6]:
''' STEP 5 - LIST ACCOUNTS '''
import requests
url = f"https://bankaccountdata.gocardless.com/api/v2/requisitions/"

headers = {
    "accept": "application/json",
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token}"  # Replace ACCESS_TOKEN with your actual access token
}

response = requests.get(url, headers=headers)
response.json()

{'count': 103,
 'next': 'https://bankaccountdata.gocardless.com/api/v2/requisitions/?limit=100&offset=100',
 'previous': None,
 'results': [{'id': '44050e25-d4f2-4482-a190-361f4c8d7e3f',
   'created': '2025-03-11T22:27:11.165218Z',
   'redirect': 'https://bankstream.io/gocardless/callback',
   'status': 'LN',
   'institution_id': 'BARCLAYS_BUKBGB22',
   'agreement': 'f8f5dfe7-5a78-4151-9fac-217c3675cdd8',
   'reference': '730304975547130800579',
   'accounts': ['5abbe86b-f321-4271-9db1-19f6fe644262',
    'f9ee4378-ecc9-45c6-aedc-d982a7329072'],
   'user_language': 'EN',
   'link': 'https://ob.gocardless.com/ob-psd2/start/ae632925-0cb3-4a41-9d26-127d4a352645/BARCLAYS_BUKBGB22',
   'ssn': None,
   'account_selection': False,
   'redirect_immediate': False},
  {'id': 'fdcb9eb7-b74c-4948-8ffa-bd982f7d2e30',
   'created': '2025-02-25T15:11:11.974291Z',
   'redirect': 'http://localhost:5190/gocardless/callback',
   'status': 'CR',
   'institution_id': 'SANTANDER_PT_TOTAPTPL',
   'agreement':

In [ ]:
schema = extract_schema(transactions)


In [ ]:
""" agreement endpoint, takes agreement id """
dot = "51880436-ed52-4274-9139-959dac2bdd8f"

### accounts endpoint takes account id. 
url = f"https://bankaccountdata.gocardless.com/api/v2/agreements/enduser/{dot}/"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {access_token}"  # Replace ACCESS_TOKEN with your actual access token
}

response = requests.get(url, headers=headers)

response.json()


### Get transaction data

In [ ]:
from app.api.transactions import export_transactions_csv

await export_transactions_csv("user_2szfxuKAsHFmNfGtZtfnu3Pjdi7")

In [ ]:
requsitions = {
    "barclays": "881a82f5-9c82-4476-a88f-34921f0f5c9c",
    "natwest": "e1849ee0-6146-4cb8-8f16-a80c1eba6bca",
}

""" new account ids are issued with each new requisition """
natwest_accounts = [
    '81451d40-c0fb-4524-9ef0-7b36f7b2cc50']

barclays_accounts = [
    'd318a85e-58c1-4249-87a2-058bb817c070',
    'f9ee4378-ecc9-45c6-aedc-d982a7329072']

### accounts endpoint takes account id. 
url = f"https://bankaccountdata.gocardless.com/api/v2/accounts/{natwest_accounts[0]}"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {access_token}"  # Replace ACCESS_TOKEN with your actual access token
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    print("Response:")
    print(response.json())
else:
    print("Error:", response.text)


In [5]:
import pandas as pd
from datetime import datetime

def get_transactions_for_account(account_id, access_token):
    """Fetch transactions for a single account"""
    url = f"https://bankaccountdata.gocardless.com/api/v2/accounts/{account_id}/transactions/"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {access_token}"
    }
    
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    return response.json()['transactions']['booked']

# # Process each bank's accounts
# for bank_name, accounts in {
#     'natwest': natwest_accounts,
#     'barclays': barclays_accounts
# }.items():
#     # Process each account for this bank
#     for account_id in accounts:
#         try:
#             # Get transactions
#             transactions = get_transactions_for_account(account_id, access_token)
            
#             # Convert to DataFrame
#             df = pd.DataFrame(transactions)
            
#             # Generate filename using bank name and last 3 chars of account_id
#             filename = f"{bank_name}_{account_id}.csv"
            
#             # Save to CSV
#             df.to_csv(filename, index=False)
#             print(f"Successfully saved {len(transactions)} transactions to {filename}")
            
#         except Exception as e:
#             print(f"Error processing account {account_id}: {str(e)}")

In [ ]:
transactions = get_transactions_for_account(natwest_accounts[0], access_token)

# Convert to DataFrame
df = pd.DataFrame(transactions)

# Generate filename using bank name and last 3 chars of account_id
filename = f"natwest_{natwest_accounts[0][:-3]}.csv"

# Save to CSV
# df.to_csv(filename, index=False)
print(f"Successfully saved {len(transactions)} transactions to {filename}")

